In [8]:
import numpy as np
from scipy.special import erf

def cumulative_saturation(A, x, alpha, r, sigma, k, N0, T, t0, sigma_t):
    numerator = np.exp(-x * alpha - r**2 / (2 * sigma**2)) * k * N0 * alpha * (
        erf((T - t0) / (np.sqrt(2) * sigma_t)) + erf(t0 / (np.sqrt(2) * sigma_t))
    )
    denominator = 4 * np.pi * sigma**2
    term = numerator / denominator
    return A * (-1 + 2 / (1 + np.exp(-term)))


def cumulative_no_saturation(x, alpha, r, sigma, N0, T, t0, sigma_t):
    return (
        np.exp(-x * alpha - r**2 / (2 * sigma**2)) 
        * N0 * alpha 
        * (erf((T - t0) / (np.sqrt(2) * sigma_t)) + erf(t0 / (np.sqrt(2) * sigma_t)))
    ) / (4 * np.pi * sigma**2)

def derivative(A, x, y, alpha, sigma_y, sigma_t, t, t0, k, N0):
    """
    Compute the given mathematical expression with the specified parameters.

    Parameters:
        A (float): Constant A
        x (float): Position x
        y (float): Position y
        alpha (float): Parameter alpha
        sigma_y (float): Standard deviation in y-direction
        sigma_t (float): Standard deviation in time
        t (float): Current time
        t0 (float): Reference time
        k (float): Parameter k
        N0 (float): Parameter N0

    Returns:
        float: Computed result
    """
    # Common terms
    exp1 = np.exp(-x * alpha - (y**2) / (2 * sigma_y**2))
    erf_term1 = erf((t - t0) / (np.sqrt(2) * sigma_t))
    erf_term2 = erf(t0 / (np.sqrt(2) * sigma_t))
    erf_sum = erf_term1 + erf_term2

    # clip1 = np.clip(1 / 4
    #         * (
    #             -4 * x * alpha
    #             - 2 * y**2 / sigma_y**2
    #             - 2 * (t - t0)**2 / sigma_t**2
    #             + (exp1 * k * N0  * alpha * erf_sum) / (np.pi * sigma_y**2)
    #         ), -100, 100)
    # Numerator
    # numerator = (
    #     A
    #     * np.exp(
    #         np.clip(1 / 4
    #         * (
    #             -4 * x * alpha
    #             - 2 * y**2 / sigma_y**2
    #             - 2 * (t - t0)**2 / sigma_t**2
    #             + (exp1 * k * N0  * alpha * erf_sum) / (np.pi * sigma_y**2)
    #         ),-100,100)
    #     )
    #     * k
    #     * N0
    #     * alpha
    # )

    numerator = (
        A
        * np.exp(
            1 / 4
            * (
                -4 * x * alpha
                - 2 * y**2 / sigma_y**2
                - 2 * (t - t0)**2 / sigma_t**2
                + (exp1 * k * N0  * alpha * erf_sum) / (np.pi * sigma_y**2)
            )
        )
        * k
        * N0
        * alpha
    )


    # Denominator
    denominator = (
        np.sqrt(2)
        * (
            1
            + np.exp(
                (exp1 * k * N0 * alpha * erf_sum) / (4 * np.pi * sigma_y**2)
            )
        )**2
        * np.pi**(3 / 2)
        * sigma_y**2
        * sigma_t
    )

    # Result
    result = numerator / denominator
    return result


In [ ]:
from dash import Dash, dcc, html, Input, Output
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

# Constants

x = np.linspace(0, 1, 100)
t0 = 0.1
sigma_t = 0.01
sigma_y = 2
alpha = 7
y = 0

# Create the Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Interactive Plot: Derivative and Cumulatives"),
    html.Div([
        html.Label("t:"),
        dcc.Slider(id='slider-t', min=0.08, max=2, step=0.1, value=0.08),
        html.Label("k:"),
        dcc.Slider(id='slider-k', min=0, max=10, step=1, value=0),
        html.Label("N0:"),
        dcc.Slider(id='slider-N0', min=0, max=10, step=0.5, value=1),
        html.Label("A:"),
        dcc.Slider(id='slider-A', min=0, max=10, step=0.5, value=1),
    ], style={'width': '50%', 'margin': 'auto'}),
    dcc.Graph(id='interactive-plot')
])

@app.callback(
    Output('interactive-plot', 'figure'),
    [Input('slider-t', 'value'),
     Input('slider-k', 'value'),
     Input('slider-N0', 'value'),
     Input('slider-A', 'value')]
)
def update_plot(t, k, N0, A):
    # Calculate values
    derivative_values = derivative(A, x, y, alpha, sigma_y, sigma_t, t, t0, k, N0)
    cum_sat_values = cumulative_saturation(A, x, alpha, y, sigma_y, k, N0, t, t0, sigma_t)
    cum_no_sat_values = cumulative_no_saturation(x, alpha, y, sigma_y, N0, t, t0, sigma_t)
    
    # Create subplots
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=("Derivative", "Cumulative Saturation and No Saturation")
    )
    
    # Add derivative plot
    fig.add_trace(
        go.Scatter(x=x, y=derivative_values, mode='lines', name='Derivative'),
        row=1, col=1
    )
    
    # Add cumulative plots
    fig.add_trace(
        go.Scatter(x=x, y=cum_sat_values, mode='lines', name='Cumulative Saturation'),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(x=x, y=cum_no_sat_values, mode='lines', name='Cumulative No Saturation'),
        row=1, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=300,
        width=800,
        showlegend=True,
        title="Interactive Derivatives and Cumulatives"
    )
    
    fig.update_xaxes(title_text="x", row=1, col=1)
    fig.update_xaxes(title_text="x", row=2, col=1)
    fig.update_yaxes(title_text="Derivative", row=1, col=1)
    fig.update_yaxes(title_text="Cumulative Values", row=2, col=1)
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
)
def update_plot(t, k):
    # Calculate values
    derivative_values = derivative(A, x, y, alpha, sigma_y, sigma_t, t, t0, k, N0)
    cum_sat_values = cumulative_saturation(A, x, alpha, y, sigma_y, k, N0, t, t0, sigma_t)
    cum_no_sat_values = cumulative_no_saturation(x, alpha, y, sigma_y, N0, t, t0, sigma_t)
    
    # Create subplots
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=("Derivative", "Cumulative Saturation and No Saturation")
    )
    
    # Add derivative plot
    fig.add_trace(
        go.Scatter(x=x, y=derivative_values, mode='lines', name='Derivative'),
        row=1, col=1
    )
    
    # Add cumulative plots
    fig.add_trace(
        go.Scatter(x=x, y=cum_sat_values, mode='lines', name='Cumulative Saturation'),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(x=x, y=cum_no_sat_values, mode='lines', name='Cumulative No Saturation'),
        row=1, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=300,
        width=800,
        showlegend=True,
        title="Interactive Derivatives and Cumulatives"
    )
    
    fig.update_xaxes(title_text="x", row=1, col=1)
    fig.update_xaxes(title_text="x", row=2, col=1)
    fig.update_yaxes(title_text="Derivative", row=1, col=1)
    fig.update_yaxes(title_text="Cumulative Values", row=2, col=1)
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
